In [1]:
best_schedule = catalog.load('best_schedule')

[07/10/24 16:27:14] INFO     Loading data from best_schedule (PickleDataset)...                 ]8;id=7226;file://C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=446601;file://C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

In [2]:
from collections import defaultdict
import random

In [44]:
# ( job_idx, task, machine, start, duration, task_num )
best_schedule


[
    (9, 1, 3, 0, 162.0, 0),
    (9, 2, 7, 162.0, 19.8, 1),
    (9, 3, 11, 181.8, 42.0, 2),
    (9, 4, 12, 223.8, 54.0, 3),
    (9, 5, 8, 277.8, 15.0, 4),
    (9, 6, 15, 292.8, 25.2, 5),
    (9, 7, 16, 318.0, 48.0, 6),
    (12, 1, 1, 0, 162.0, 0),
    (12, 2, 9, 162.0, 19.8, 1),
    (12, 3, 11, 223.8, 42.0, 2),
    (12, 4, 13, 265.8, 54.0, 3),
    (12, 5, 10, 319.8, 15.0, 4),
    (12, 6, 15, 334.8, 25.2, 5),
    (12, 7, 17, 360.0, 48.0, 6),
    (23, 1, 2, 0, 150.0, 0),
    (23, 2, 7, 181.8, 19.8, 1),
    (23, 3, 11, 265.8, 42.0, 2),
    (23, 4, 14, 307.8, 54.0, 3),
    (23, 5, 9, 361.8, 15.0, 4),
    (23, 6, 15, 376.8, 25.2, 5),
    (23, 7, 16, 402.0, 48.0, 6),
    (1, 1, 4, 0, 162.0, 0),
    (1, 2, 7, 201.60000000000002, 19.8, 1),
    (1, 3, 11, 307.8, 42.0, 2),
    (1, 4, 12, 349.8, 54.0, 3),
    (1, 5, 7, 403.8, 15.0, 4),
    (1, 6, 15, 418.8, 25.2, 5),
    (1, 7, 17, 444.0, 48.0, 6),
    (5, 1, 5, 0, 162.0, 0),
    (5, 2, 8, 292.8, 19.8, 1),
    (5, 3, 11, 349.8, 42.0, 2),
    (5

In [45]:
best_sched_copy = best_schedule.copy()

In [46]:
best_sched_copy


[
    (9, 1, 3, 0, 162.0, 0),
    (9, 2, 7, 162.0, 19.8, 1),
    (9, 3, 11, 181.8, 42.0, 2),
    (9, 4, 12, 223.8, 54.0, 3),
    (9, 5, 8, 277.8, 15.0, 4),
    (9, 6, 15, 292.8, 25.2, 5),
    (9, 7, 16, 318.0, 48.0, 6),
    (12, 1, 1, 0, 162.0, 0),
    (12, 2, 9, 162.0, 19.8, 1),
    (12, 3, 11, 223.8, 42.0, 2),
    (12, 4, 13, 265.8, 54.0, 3),
    (12, 5, 10, 319.8, 15.0, 4),
    (12, 6, 15, 334.8, 25.2, 5),
    (12, 7, 17, 360.0, 48.0, 6),
    (23, 1, 2, 0, 150.0, 0),
    (23, 2, 7, 181.8, 19.8, 1),
    (23, 3, 11, 265.8, 42.0, 2),
    (23, 4, 14, 307.8, 54.0, 3),
    (23, 5, 9, 361.8, 15.0, 4),
    (23, 6, 15, 376.8, 25.2, 5),
    (23, 7, 16, 402.0, 48.0, 6),
    (1, 1, 4, 0, 162.0, 0),
    (1, 2, 7, 201.60000000000002, 19.8, 1),
    (1, 3, 11, 307.8, 42.0, 2),
    (1, 4, 12, 349.8, 54.0, 3),
    (1, 5, 7, 403.8, 15.0, 4),
    (1, 6, 15, 418.8, 25.2, 5),
    (1, 7, 17, 444.0, 48.0, 6),
    (5, 1, 5, 0, 162.0, 0),
    (5, 2, 8, 292.8, 19.8, 1),
    (5, 3, 11, 349.8, 42.0, 2),
    (5

In [39]:
import random
from collections import defaultdict

def mutate(schedule):
    # Group tasks by job_idx
    jobs = defaultdict(list)
    for task in schedule:
        jobs[task[0]].append(task)
    
    # Find pairs of jobs with same number of tasks and same durations
    job_pairs = []
    job_list = list(jobs.items())
    
    for i in range(len(job_list)):
        for j in range(i + 1, len(job_list)):
            job1, tasks1 = job_list[i]
            job2, tasks2 = job_list[j]
            
            if len(tasks1) == len(tasks2):
                durations1 = [task[4] for task in tasks1]
                durations2 = [task[4] for task in tasks2]
                
                if durations1 == durations2:
                    job_pairs.append((job1, job2))
    
    # If no pairs found, return schedule unchanged
    if not job_pairs:
        return schedule
    
    # Randomly select a pair of jobs
    job1, job2 = random.choice(job_pairs)
    
    # Swap the start times of the tasks in the selected jobs
    tasks1 = jobs[job1]
    tasks2 = jobs[job2]
    
    for i in range(len(tasks1)):
        task1 = tasks1[i]
        task2 = tasks2[i]
        
        # Create new tasks with swapped start times
        new_task1 = (task1[0], task1[1], task1[2], task2[3], task1[4], task1[5])
        new_task2 = (task2[0], task2[1], task2[2], task1[3], task2[4], task2[5])
        
        # Update schedule
        schedule[schedule.index(task1)] = new_task1
        schedule[schedule.index(task2)] = new_task2
    
    return schedule

# Example schedule
schedule = [
    (30, 1, 6, 684.0, 360.0, 0),
    (30, 2, 10, 1090.8, 19.8, 1),
    (30, 3, 11, 1351.8, 48.0, 2),
    (30, 6, 15, 1399.8, 25.2, 3),
    (30, 7, 16, 1425.0, 48.0, 4),
    (31, 1, 2, 924.0, 360.0, 0),
    (31, 2, 9, 1284.0, 19.8, 1),
    (31, 3, 11, 1303.8, 48.0, 2),
    (31, 6, 15, 1351.8, 25.2, 3),
    (31, 7, 17, 1377.0, 48.0, 4)
]

# Perform mutation
new_schedule = mutate(schedule)

# Display the new schedule
print(new_schedule)


[(30, 1, 6, 924.0, 360.0, 0), (30, 2, 10, 1284.0, 19.8, 1), (30, 3, 11, 1303.8, 48.0, 2), (30, 6, 15, 1351.8, 25.2, 3), (30, 7, 16, 1377.0, 48.0, 4), (31, 1, 2, 684.0, 360.0, 0), (31, 2, 9, 1090.8, 19.8, 1), (31, 3, 11, 1351.8, 48.0, 2), (31, 6, 15, 1399.8, 25.2, 3), (31, 7, 17, 1425.0, 48.0, 4)]


In [40]:
schedule


[
    (30, 1, 6, 924.0, 360.0, 0),
    (30, 2, 10, 1284.0, 19.8, 1),
    (30, 3, 11, 1303.8, 48.0, 2),
    (30, 6, 15, 1351.8, 25.2, 3),
    (30, 7, 16, 1377.0, 48.0, 4),
    (31, 1, 2, 684.0, 360.0, 0),
    (31, 2, 9, 1090.8, 19.8, 1),
    (31, 3, 11, 1351.8, 48.0, 2),
    (31, 6, 15, 1399.8, 25.2, 3),
    (31, 7, 17, 1425.0, 48.0, 4)
]

In [41]:
new_schedule


[
    (30, 1, 6, 924.0, 360.0, 0),
    (30, 2, 10, 1284.0, 19.8, 1),
    (30, 3, 11, 1303.8, 48.0, 2),
    (30, 6, 15, 1351.8, 25.2, 3),
    (30, 7, 16, 1377.0, 48.0, 4),
    (31, 1, 2, 684.0, 360.0, 0),
    (31, 2, 9, 1090.8, 19.8, 1),
    (31, 3, 11, 1351.8, 48.0, 2),
    (31, 6, 15, 1399.8, 25.2, 3),
    (31, 7, 17, 1425.0, 48.0, 4)
]

In [35]:
updated_schedule = mutate(best_schedule)

First job: 30
Second job: 31


In [36]:
def filter_schedule_by_job(schedule, job_idx):
    return [task for task in schedule if task[0] in job_idx]

In [47]:
filt_old = filter_schedule_by_job(best_sched_copy, [30, 31])
filt_old


[
    (30, 1, 6, 924.0, 360.0, 0),
    (30, 2, 10, 1284.0, 19.8, 1),
    (30, 3, 11, 1303.8, 48.0, 2),
    (30, 6, 15, 1351.8, 25.2, 3),
    (30, 7, 16, 1377.0, 48.0, 4),
    (31, 1, 2, 684.0, 360.0, 0),
    (31, 2, 9, 1090.8, 19.8, 1),
    (31, 3, 11, 1351.8, 48.0, 2),
    (31, 6, 15, 1399.8, 25.2, 3),
    (31, 7, 17, 1425.0, 48.0, 4)
]

In [48]:
filt_new = filter_schedule_by_job(updated_schedule, [30, 31])
filt_new


[
    (30, 1, 6, 684.0, 360.0, 0),
    (30, 2, 10, 1090.8, 19.8, 1),
    (30, 3, 11, 1351.8, 48.0, 2),
    (30, 6, 15, 1399.8, 25.2, 3),
    (30, 7, 16, 1425.0, 48.0, 4),
    (31, 1, 2, 924.0, 360.0, 0),
    (31, 2, 9, 1284.0, 19.8, 1),
    (31, 3, 11, 1303.8, 48.0, 2),
    (31, 6, 15, 1351.8, 25.2, 3),
    (31, 7, 17, 1377.0, 48.0, 4)
]

In [ ]:
# Test with and without mutate:
# 1500 inds, 40 generations
without_mutate = [-758, -1024, -542, 809, 2041, -2194, 2210]
with_mutate = [8624, 9333, 7330, 8275, 6319, 5015, 4996]

### Load processed orders

In [3]:
processed_orders = catalog.load('croom_processed_orders')

[07/10/24 16:28:05] INFO     Loading data from croom_processed_orders (ParquetDataset)...       ]8;id=446656;file://C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py\data_catalog.py]8;;\:]8;id=926515;file://C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\kedro\io\data_catalog.py#483\483]8;;\

In [8]:
processed_orders.loc[22, :]

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\pandas\core\indexes\base │
│ .py:3805 in get_loc                                                                              │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:167                                                    │
│                                                                                                  │
│ in pandas._libs.index.IndexEngine.get_loc:196                                                    │
│                                                                                                  │
│ in pandas._libs.hashtable.Int64HashTable.get_item:2606                                           │
│                                                                                                  │
│ in pandas._libs.hashtable.Int64HashTable.get_item:2630                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 22

The above exception was the direct cause of the following exception:

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 processed_orders.loc[22, :]                                                                  │
│   2                                                                                              │
│                                                                                                  │
│ C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\pandas\core\indexing.py: │
│ 1184 in __getitem__                                                                              │
│                                                                                                  │
│ C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\pandas\core\indexing.py: │
│ 1368 in _getitem_tuple                                                                           │
│                                                                                                  │
│ C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\pandas\core\indexing.py: │
│ 1065 in _getitem_lowerdim                                                                        │
│                                                                                                  │
│ C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\pandas\core\indexing.py: │
│ 1431 in _getitem_axis                                                                            │
│                                                                                                  │
│ C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\pandas\core\indexing.py: │
│ 1381 in _get_label                                                                               │
│                                                                                                  │
│ C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\pandas\core\generic.py:4 │
│ 301 in xs                                                                                        │
│                                                                                                  │
│ C:\Users\JeanLucOudshoorn\miniconda3\envs\EIT-Epsilon\Lib\site-packages\pandas\core\indexes\base │
│ .py:3812 in get_loc                                                                              │
╰─────────────────────────────────────────────────────────────────────────────

In [11]:
processed_orders

,Job ID,Created Date,Created By,Prod Due Date,Customer ID,Customer Name,Leadtime Target,To Sort,Due Date,Overdue Y/N,...,Production Qty,Planning Complete?,Schedule Complete?,Released to Floor?,On Hold?,Days_Open,Type,Size,Orientation,Cementless
0,4421322,2024-03-01 07:40:55,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
1,4421321,2024-03-01 07:39:52,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
2,4421320,2024-03-01 07:38:43,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
3,4421319,2024-03-01 07:37:21,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
4,4421318,2024-03-01 07:36:12,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
5,4421317,2024-03-01 07:33:56,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
6,4421316,2024-03-01 07:26:07,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
7,4421315,2024-03-01 07:22:15,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
8,4421314,2024-03-01 07:21:15,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
9,4421313,2024-03-01 07:20:17,Elaine,2024-03-10,322,DEPUY SYNTHES,15,1,2024-03-22,1,...,12,False,False,True,False,0,CR,8,LEFT,CLS
